# Policy Evaluation, Policy Improvement and Policy Iteration 

In this notebook, we will create a Frozen lake environment using Gym library and then we will write policy evaluation, policy improvement and policy iteration algorithms from scratch and train the frozen lake enviroment with a random policy to obtain optimal policy.

# Frozen Lake Environment

In [1]:
import warnings ; warnings.filterwarnings('ignore')

import gym
import numpy as np
from pprint import pprint
from tqdm import tqdm_notebook as tqdm

from itertools import cycle

import random

np.set_printoptions(suppress=True)
random.seed(123); np.random.seed(123)

In [2]:
env = gym.make('FrozenLake-v0')
P = env.env.P
init_state = env.reset()
goal_state = 15

LEFT, DOWN, RIGHT, UP = range(4)
random_pi = lambda s: {
    0:RIGHT, 1:LEFT, 2:DOWN, 3:UP,
    4:LEFT, 5:LEFT, 6:RIGHT, 7:LEFT,
    8:UP, 9:DOWN, 10:UP, 11:LEFT,
    12:LEFT, 13:RIGHT, 14:DOWN, 15:LEFT
}[s]

# Helper Functions

In [7]:
def print_policy(pi, P, action_symbols=('<', 'v', '>', '^'), n_cols=4, title='Policy:'):
    print(title)
    arrs = {k:v for k,v in enumerate(action_symbols)}
    for s in range(len(P)):
        a = pi(s)
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), arrs[a].rjust(6), end=" ")
        if (s + 1) % n_cols == 0: print("|")

In [8]:
def print_state_value_function(V, P, n_cols=4, prec=3, title='State-value function:'):
    print(title)
    for s in range(len(P)):
        v = V[s]
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), '{}'.format(np.round(v, prec)).rjust(6), end=" ")
        if (s + 1) % n_cols == 0: print("|")

In [9]:
def print_action_value_function(Q, 
                                optimal_Q=None, 
                                action_symbols=('<', '>'), 
                                prec=3, 
                                title='Action-value function:'):
    vf_types=('',) if optimal_Q is None else ('', '*', 'err')
    headers = ['s',] + [' '.join(i) for i in list(itertools.product(vf_types, action_symbols))]
    print(title)
    states = np.arange(len(Q))[..., np.newaxis]
    arr = np.hstack((states, np.round(Q, prec)))
    if not (optimal_Q is None):
        arr = np.hstack((arr, np.round(optimal_Q, prec), np.round(optimal_Q-Q, prec)))
    print(tabulate(arr, headers, tablefmt="fancy_grid"))

In [10]:
def probability_success(env, pi, goal_state, n_episodes=100, max_steps=200):
    random.seed(123); np.random.seed(123) ; env.seed(123)
    results = []
    for _ in range(n_episodes):
        state, done, steps = env.reset(), False, 0
        while not done and steps < max_steps:
            state, _, done, h = env.step(pi(state))
            steps += 1
        results.append(state == goal_state)
    return np.sum(results)/len(results)

In [11]:
def mean_return(env, pi, n_episodes=100, max_steps=200):
    random.seed(123); np.random.seed(123) ; env.seed(123)
    results = []
    for _ in range(n_episodes):
        state, done, steps = env.reset(), False, 0
        results.append(0.0)
        while not done and steps < max_steps:
            state, reward, done, _ = env.step(pi(state))
            results[-1] += reward
            steps += 1
    return np.mean(results)

# Checking Frozen Lake Environment

In [12]:
print_policy(random_pi, P)

Policy:
| 00      > | 01      < | 02      v | 03      ^ |
| 04      < |           | 06      > |           |
| 08      ^ | 09      v | 10      ^ |           |
|           | 13      > | 14      v |           |


In [13]:
print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, random_pi, goal_state=goal_state)*100, 
    mean_return(env, random_pi)))

Reaches goal 12.00%. Obtains an average undiscounted return of 0.1200.


# Policy Evaluation

In [16]:
def policy_evaluation(pi, P, gamma=1.0, theta = 1e-10):
    prev_V = np.zeros(len(P), dtype=np.float64)
    while True:
        V = np.zeros(len(P), dtype=np.float64)
        for s in range (len(P)):
            for prob, next_state, reward, done in P[s][pi(s)]:
                V[s]+= prob * (reward + gamma * prev_V[next_state] * (not done))
        if np.max(np.abs(prev_V - V)) < theta:
            break
        prev_V = V.copy()
    return V
                

In [21]:
V = policy_evaluation(random_pi, P, gamma=0.99)
V

array([0.09554433, 0.04705915, 0.0470064 , 0.04562386, 0.1469248 ,
       0.        , 0.04976062, 0.        , 0.20275753, 0.26473443,
       0.10378337, 0.        , 0.        , 0.49568466, 0.74165563,
       0.        ])

In [22]:
print_state_value_function(V, P, n_cols=4, prec=4)

State-value function:
| 00 0.0955 | 01 0.0471 | 02  0.047 | 03 0.0456 |
| 04 0.1469 |           | 06 0.0498 |           |
| 08 0.2028 | 09 0.2647 | 10 0.1038 |           |
|           | 13 0.4957 | 14 0.7417 |           |


# Policy Improvement

In [24]:
def policy_improvement(V, P, gamma=1.0):
    Q = np.zeros((len(P),len(P[0])), dtype=np.float64)
    for s in range(len(P)):
        for a in range (len(P[s])):
            for prob, next_state, reward, done in P[s][a]:
                Q[s][a] += prob * (reward + gamma * V[next_state] * (not done))
    new_pi = lambda s : {s:a for s,a in enumerate(np.argmax(Q, axis=1))}[s]
    return new_pi

In [25]:
random_plus_pi = policy_improvement(V, P, gamma=0.99)
print_policy(random_plus_pi, P)
print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, random_plus_pi, goal_state=goal_state)*100, 
    mean_return(env, random_plus_pi)))

Policy:
| 00      < | 01      ^ | 02      < | 03      ^ |
| 04      < |           | 06      < |           |
| 08      ^ | 09      v | 10      < |           |
|           | 13      > | 14      v |           |
Reaches goal 74.00%. Obtains an average undiscounted return of 0.7400.


In [26]:
new_V = policy_evaluation(random_plus_pi, P, gamma=0.99)
print_state_value_function(new_V, P, prec=4)

State-value function:
| 00 0.5325 | 01 0.4498 | 02 0.3807 | 03 0.3695 |
| 04 0.5486 |           | 06 0.3232 |           |
| 08 0.5814 | 09 0.6318 | 10 0.5987 |           |
|           | 13 0.7344 | 14 0.8592 |           |


In [27]:
print_state_value_function(new_V - V, P, prec=4)

State-value function:
| 00 0.4369 | 01 0.4027 | 02 0.3337 | 03 0.3239 |
| 04 0.4017 |           | 06 0.2734 |           |
| 08 0.3786 | 09  0.367 | 10 0.4949 |           |
|           | 13 0.2387 | 14 0.1176 |           |


# Policy Iteration

In [28]:
def policy_iteration(P, gamma=1.0, theta=1e-10):
    random_actions = np.random.choice(tuple(P[0].keys()),len(P))
    pi = lambda s : {s:a for s,a in enumerate(random_actions)}[s]
    
    while True:
        old_pi = {s:pi(s) for s in range(len(P))}
        V = policy_evaluation(pi, P, gamma, theta)
        pi = policy_improvement(V, P, gamma)
        if old_pi == {s:pi(s) for s in range(len(P))}:
            break
    return V, pi

In [29]:
V_best_p, pi_best_p = policy_iteration(P, gamma=0.99)
print_state_value_function(V_best_p, P, prec=4)
print()
print('Optimal policy and state-value function (PI):')
print_policy(pi_best_p, P)
print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, pi_best_p, goal_state=goal_state)*100, 
    mean_return(env, pi_best_p)))

State-value function:
| 00  0.542 | 01 0.4988 | 02 0.4707 | 03 0.4569 |
| 04 0.5585 |           | 06 0.3583 |           |
| 08 0.5918 | 09 0.6431 | 10 0.6152 |           |
|           | 13 0.7417 | 14 0.8628 |           |

Optimal policy and state-value function (PI):
Policy:
| 00      < | 01      ^ | 02      ^ | 03      ^ |
| 04      < |           | 06      < |           |
| 08      ^ | 09      v | 10      < |           |
|           | 13      > | 14      v |           |
Reaches goal 74.00%. Obtains an average undiscounted return of 0.7400.
